# OrbitalElements interactive examples

## Working environment & Packages

Set the working environment to the repository root.

In [ ]:
using Plots
plotlyjs();

In [ ]:
using Pkg
Pkg.activate(@__DIR__()*"/../.")
import OrbitalElements as OE

# Generic parameters

## Potential

In [ ]:
G, M, bc = 1.0, 1.0, 1.0
ψ(r::Float64)   = OE.ψIsochrone(r,bc,M,G)
dψ(r::Float64)  = OE.dψIsochrone(r,bc,M,G)
d2ψ(r::Float64) = OE.d2ψIsochrone(r,bc,M,G)
Ω₀ = OE.Ω₀Isochrone(bc,M,G);

#G = 1.0
#R0, V0 = 20., 1.
#ψ(r::Float64)   = OE.ψMestel(r,R0,V0)
#dψ(r::Float64)  = OE.dψMestel(r,R0,V0)
#d2ψ(r::Float64) = OE.d2ψMestel(r,R0,V0)
#Ω₀ = OE.Ω₀Mestel(R0,V0);

## Parameters

In [ ]:
# Radii for frequency truncations
rmin, rmax = 0.1, 100.0
# Tolerances / Taylor expansions
EDGE = 0.01
TOLECC = 0.01
TOLA = 1.0
NINT = 32

const fixedparams = OE.OrbitalParameters(Ω₀=Ω₀,
                                            rmin=rmin,rmax=rmax);
                                            #EDGE=EDGE,TOLA=TOLA,TOLECC=TOLECC,
                                            #NINT=NINT);

# Frequency computation

Frequency values along a constant eccentricity line. Example using the isochrone potential for which analytical expression of the frequencies are known. Comparing analytical and numerical results.

In [ ]:
function FrequenciesE!(a::Float64,
                      tabe::Vector{Float64},
                      results::Array{Float64})
    @assert size(results) == (2,length(tabe))
    for j=1:ne
        Ω1, Ω2 = OE.ComputeFrequenciesAE(ψ,dψ,d2ψ,a,tabe[j],fixedparams)
        @inbounds results[1,j], results[2,j] = Ω1, Ω2
    end
end;
function IsochroneFrequenciesE!(a::Float64,
                      tabe::Vector{Float64},
                      results::Array{Float64})
    @assert size(results) == (2,length(tabe))
    for j=1:ne
        Ω1true, Ω2true = OE.IsochroneOmega12FromAE(a,tabe[j])
        @inbounds results[1,j], results[2,j] = Ω1true, Ω2true
    end
end;

In [ ]:
ne = 100;
tabe = collect(LinRange(0.,1.,ne));
tabΩ = zeros(2,ne);
tabΩIso = zeros(2,ne);

In [ ]:
a = 1.0

# Compute
FrequenciesE!(a,tabe,tabΩ)
IsochroneFrequenciesE!(a,tabe,tabΩIso)

# Plot
xlabelΩ = ["e" "e"]
ylabelΩ = ["Ω1" "Ω2"]
p1 = plot(tabe,tabΩ[1,:],xlabel=xlabelΩ[1],ylabel=ylabelΩ[1],label="Numerical");
plot!(tabe,tabΩIso[1,:],label="True")
p2 = plot(tabe,tabΩ[2,:],xlabel=xlabelΩ[2],ylabel=ylabelΩ[2],label=false);
plot!(tabe,tabΩIso[2,:],label=false)
plot(p1,p2,layout=(1,2),size=(900,300))

# Frequency derivatives computation

Frequency values along a constant eccentricity line. Example using the isochrone potential for which analytical expression of the frequencies are known. Comparing analytical and numerical results.

In [ ]:
function FrequenciesDerivE!(a::Float64,
                      tabe::Vector{Float64},
                      results::Array{Float64})
    @assert size(results) == (6,length(tabe))
    for j=1:ne
        Ω1, Ω2, ∂Ω1∂a, ∂Ω2∂a, ∂Ω1∂e, ∂Ω2∂e = OE.ComputeFrequenciesAEWithDeriv(ψ,dψ,d2ψ,a,tabe[j],fixedparams)
        @inbounds results[1,j], results[2,j], results[3,j], results[4,j], results[5,j], results[6,j]= Ω1, Ω2, ∂Ω1∂a, ∂Ω2∂a, ∂Ω1∂e, ∂Ω2∂e
    end
end;

In [ ]:
ne = 100;
tabe = collect(LinRange(0.,1.,ne));
tabderΩ = zeros(6,ne);

In [ ]:
a = 1.0

# Compute
FrequenciesDerivE!(a,tabe,tabderΩ)

# Plot
xlabelderΩ = ["" "" "" "" "e" "e"]
ylabelderΩ = ["Ω1" "Ω2" "∂Ω1∂a" "∂Ω2∂a" "∂Ω1∂e" "∂Ω2∂e"]
plot(tabe,transpose(tabderΩ),layout=(3,2),xlabel=xlabelderΩ,ylabel=ylabelderΩ,legend=false,size=(900,600))

# Energy, angular momentum and derivatives computation

In [ ]:
function ELDerivE!(a::Float64,
                      tabe::Vector{Float64},
                      results::Array{Float64})
    @assert size(results) == (6,length(tabe))
    for j=1:ne
        E, L, ∂E∂a, ∂L∂a, ∂E∂e, ∂L∂e = OE.dELFromAE(ψ,dψ,d2ψ,a,tabe[j],fixedparams)
        @inbounds results[1,j], results[2,j], results[3,j], results[4,j], results[5,j], results[6,j]= E, L, ∂E∂a, ∂L∂a, ∂E∂e, ∂L∂e
    end
end;

In [ ]:
ne = 100;
tabe = collect(LinRange(0.,1.,ne));
tabderEL = zeros(6,ne);

In [ ]:
a = 1.0

# Compute
ELDerivE!(a,tabe,tabderEL)

# Plot
xlabelEL = ["" "" "" "" "e" "e"]
ylabelEL = ["E" "L" "∂E∂a" "∂L∂a" "∂E∂e" "∂L∂e"]
plot(tabe,transpose(tabderEL),layout=(3,2),xlabel=xlabelEL,ylabel=ylabelEL,legend=false,size=(900,600))